<a href="https://colab.research.google.com/github/marcelofschiavo/sql-people-analytics/blob/main/people_analytics_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfólio SQL: Análise de People Analytics

**Autor:** Marcelo Ferreira Schiavo [LinkedIn](https://www.linkedin.com/in/marceloschiavo/) | [GitHub](https://github.com/marcelofschiavo) <br>
**Objetivo:** Este notebook documenta um projeto de análise de dados ponta-a-ponta, demonstrando a aplicação de consultas SQL (do básico ao avançado) para extrair insights de negócio de um conjunto de dados de Recursos Humanos (People Analytics).

**Etapas:**
1. Levantamento dos problemas e perguntas de negócio
2. Tratamento dos dados brutos (ETL/ELT e EDA)
3. Análise e Modelagem
4. Geração de insights para decisão (Storytelling).

**Stack de Ferramentas:**
- **Google Colab / Notebook Jupyter:** Ambiente interativo de análise exploratória e documentação.
- **Python:** Linguagem base para orquestração de dados e setup do ambiente.
- **DuckDB:** BD/Motor SQL com foco analítico (OLAP) para consultas de alta performance diretamente nos arquivos `.csv`.
- **Pandas:** Utilizado para a exibição e visualização final dos DataFrames resultantes das consultas.

In [ ]:
# CÉLULA DE SETUP
# 1. Importar as bibliotecas
# !pip install duckdb pandas
import duckdb
import pandas as pd
from IPython.display import display # Para exibir os DataFrames

print("DuckDB e Pandas prontos.")

DuckDB e Pandas prontos.


## Seção 1: Setup do Ambiente e Carga de Dados (ETL)
O processo de análise se inicia com a obtenção e preparação dos dados. Para garantir a reprodutibilidade deste notebook, os dados brutos (que normalmente seriam arquivos `.csv` independentes) serão simulados e escritos em arquivos locais.

*(Esta seção pode ser recolhida, pois seu foco é apenas o setup).*

In [ ]:
# Criando Departamentos.csv
csv_departamentos = """ID_Departamento,Nome_Departamento,Localizacao
1,Engenharia,São Paulo
2,Vendas,Rio de Janeiro
3,Marketing,São Paulo
4,Recursos Humanos,Belo Horizonte
5,Produto,São Paulo
"""

# Criando Funcionarios.csv
csv_funcionarios = """ID_Funcionario,Nome,Sobrenome,Email,Data_Contratacao,Data_Nascimento,Genero,Salario_Anual,Cargo,ID_Departamento,ID_Gerente
101,Ana,Silva,ana.silva@empresa.com,2019-03-15,1985-07-20,Feminino,150000.00,Gerente de Engenharia,1,
102,Bruno,Costa,bruno.costa@empresa.com,2018-05-01,1982-11-10,Masculino,140000.00,Gerente de Vendas,2,
103,Carla,Dias,carla.dias@empresa.com,2020-01-10,1990-02-25,Feminino,110000.00,Gerente de Marketing,3,
104,Daniel,Ferreira,daniel.ferreira@empresa.com,2021-02-20,1992-04-12,Masculino,90000.00,Engenheiro de Software Pleno,1,101
105,Elisa,Gomes,elisa.gomes@empresa.com,2022-07-30,1995-09-05,Feminino,75000.00,Engenheira de Software Júnior,1,101
106,Fábio,Martins,fabio.martins@empresa.com,2019-11-05,1988-12-19,Masculino,120000.00,Engenheiro de Software Sênior,1,101
107,Gabriela,Nunes,gabriela.nunes@empresa.com,2020-08-14,1993-01-30,Feminino,85000.00,Representante de Vendas Sênior,2,102
108,Heitor,Oliveira,heitor.oliveira@empresa.com,2022-01-10,1996-06-15,Masculino,65000.00,Representante de Vendas Júnior,2,102
109,Isabela,Pereira,isabela.pereira@empresa.com,2021-05-12,1994-03-22,Feminino,70000.00,Analista de Marketing Digital,3,103
110,Joana,Rodrigues,joana.rodrigues@empresa.com,2023-03-01,1998-08-08,Feminino,55000.00,Estagiária de Marketing,3,103
111,Lucas,Santos,lucas.santos@empresa.com,2020-09-01,1991-10-27,Masculino,95000.00,Analista de RH Sênior,4,
112,Maria,Souza,maria.souza@empresa.com,2021-11-15,1990-07-14,Feminino,130000.00,Product Manager,5,101
"""

# Criando Avaliacoes_Desempenho.csv
csv_avaliacoes = """ID_Avaliacao,ID_Funcionario,Ano_Avaliacao,Nota_Desempenho
1,101,2023,5
2,102,2023,4
3,103,2023,5
4,104,2022,3
5,104,2023,4
6,105,2023,4
7,106,2022,5
8,106,2023,5
9,107,2022,5
10,107,2023,3
11,108,2023,3
12,109,2022,4
13,109,2023,4
14,111,2023,4
15,112,2022,4
16,112,2023,5
"""

try:
    # 1. Tentando escrever Departamentos.csv
    with open('Departamentos.csv', 'w', encoding='utf-8') as f:
        f.write(csv_departamentos)

    # 2. Tentando escrever Funcionarios.csv
    with open('Funcionarios.csv', 'w', encoding='utf-8') as f:
        f.write(csv_funcionarios)

    # 3. Tentando escrever Avaliacoes_Desempenho.csv
    with open('Avaliacoes_Desempenho.csv', 'w', encoding='utf-8') as f:
        f.write(csv_avaliacoes)

except Exception as e:
    # Se QUALQUER uma das operações acima falhar, este bloco será executado
    print(f"--- ERRO ---")
    print(f"Falha ao tentar escrever os arquivos .csv.")
    print(f"Detalhe do erro: {e}")
    print("---------------------------------------------------------------")
    print("Verifique se o ambiente (Colab/Jupyter) tem permissão de escrita.")

else:
    # Este bloco 'else' SÓ é executado se o bloco 'try' inteiro for bem-sucedido
    print("Arquivos Departamentos.csv, Funcionarios.csv, e Avaliacoes_Desempenho.csv criados com sucesso.")

Arquivos Departamentos.csv, Funcionarios.csv, e Avaliacoes_Desempenho.csv criados com sucesso.


## Seção 2: Entendimento e Exploração dos Dados (EDA)

O primeiro passo em qualquer análise é inspecionar os dados. Vamos extrair uma amostra de cada tabela para entender as colunas e os tipos de dados disponíveis.

In [ ]:
# Tabela 1: Funcionarios (Amostra)
query = "SELECT * FROM 'Funcionarios.csv'"
display(duckdb.query(query).df())

# Tabela 2: Departamentos (Completa)
query = "SELECT * FROM 'Departamentos.csv'"
display(duckdb.query(query).df())

# Tabela 3: Avaliacoes_Desempenho (Amostra)
query = "SELECT * FROM 'Avaliacoes_Desempenho.csv'"
display(duckdb.query(query).df())

,ID_Funcionario,Nome,Sobrenome,Email,Data_Contratacao,Data_Nascimento,Genero,Salario_Anual,Cargo,ID_Departamento,ID_Gerente
0,101,Ana,Silva,ana.silva@empresa.com,2019-03-15,1985-07-20,Feminino,150000.0,Gerente de Engenharia,1,<NA>
1,102,Bruno,Costa,bruno.costa@empresa.com,2018-05-01,1982-11-10,Masculino,140000.0,Gerente de Vendas,2,<NA>
2,103,Carla,Dias,carla.dias@empresa.com,2020-01-10,1990-02-25,Feminino,110000.0,Gerente de Marketing,3,<NA>
3,104,Daniel,Ferreira,daniel.ferreira@empresa.com,2021-02-20,1992-04-12,Masculino,90000.0,Engenheiro de Software Pleno,1,101
4,105,Elisa,Gomes,elisa.gomes@empresa.com,2022-07-30,1995-09-05,Feminino,75000.0,Engenheira de Software Júnior,1,101
5,106,Fábio,Martins,fabio.martins@empresa.com,2019-11-05,1988-12-19,Masculino,120000.0,Engenheiro de Software Sênior,1,101
6,107,Gabriela,Nunes,gabriela.nunes@empresa.com,2020-08-14,1993-01-30,Feminino,85000.0,Representante de Vendas Sênior,2,102
7,108,Heitor,Oliveira,heitor.oliveira@empresa.com,2022-01-10,1996-06-15,Masculino,65000.0,Representante de Vendas Júnior,2,102
8,109,Isabela,Pereira,isabela.pereira@empresa.com,2021-05-12,1994-03-22,Feminino,70000.0,Analista de Marketing Digital,3,103
9,110,Joana,Rodrigues,joana.rodrigues@empresa.com,2023-03-01,1998-08-08,Feminino,55000.0,Estagiária de Marketing,3,103


,ID_Departamento,Nome_Departamento,Localizacao
0,1,Engenharia,São Paulo
1,2,Vendas,Rio de Janeiro
2,3,Marketing,São Paulo
3,4,Recursos Humanos,Belo Horizonte
4,5,Produto,São Paulo


,ID_Avaliacao,ID_Funcionario,Ano_Avaliacao,Nota_Desempenho
0,1,101,2023,5
1,2,102,2023,4
2,3,103,2023,5
3,4,104,2022,3
4,5,104,2023,4
5,6,105,2023,4
6,7,106,2022,5
7,8,106,2023,5
8,9,107,2022,5
9,10,107,2023,3


## Seção 3: Análise de Dados via SQL (Respondendo Perguntas de Negócio)
Nesta seção, cada consulta SQL é projetada para responder a uma pergunta de negócio específica. A complexidade das consultas aumentará progressivamente para demonstrar o funil de análise, desde a seleção de dados brutos até a geração de insights comparativos complexos.

### Nível 1: Filtragem e Segmentação de Dados (SELECT, WHERE, ORDER BY)
**Análise:** O primeiro nível de investigação consiste em filtrar o *dataset* para isolar subconjuntos de interesse e realizar uma inspeção básica.

* **Perguntas de Negócio:**
    1.  (Segmentação de Cargo) Quais funcionários no departamento de Engenharia se qualificam para uma revisão salarial sênior (salário >= $90k)?
    2.  (Auditoria de Compensação) Existem discrepâncias salariais evidentes ao ordenar os funcionários de um mesmo departamento por salário?

In [ ]:
query = """
SELECT
    Nome,
    Sobrenome,
    Cargo,
    Salario_Anual
FROM
    'Funcionarios.csv'
WHERE
    ID_Departamento = 1 -- ID de Engenharia
    AND Salario_Anual >= 90000
ORDER BY
    Salario_Anual DESC;
"""
display(duckdb.query(query).df())

,Nome,Sobrenome,Cargo,Salario_Anual
0,Ana,Silva,Gerente de Engenharia,150000.0
1,Fábio,Martins,Engenheiro de Software Sênior,120000.0
2,Daniel,Ferreira,Engenheiro de Software Pleno,90000.0


### Nível 2: Métricas Agregadas (GROUP BY, COUNT, AVG, SUM)
**Análise:** A agregação de dados é usada para resumir informações em macro-indicadores e calcular métricas-chave (KPIs) em nível departamental, permitindo uma visão gerencial.

* **Perguntas de Negócio:**
    1.  (Análise de Custo) Qual é a distribuição da folha de pagamento (custo total) entre os diferentes departamentos?
    2.  (Benchmarking Interno) Como o salário médio de Engenharia se compara ao de Vendas? Estamos alocando recursos de forma equilibrada?

In [ ]:
query = """
SELECT
    d.Nome_Departamento,
    COUNT(f.ID_Funcionario) AS Numero_de_Funcionarios,
    AVG(f.Salario_Anual) AS Salario_Medio_Anual,
    SUM(f.Salario_Anual) AS Folha_Pagamento_Total
FROM
    'Funcionarios.csv' AS f
INNER JOIN
    'Departamentos.csv' AS d ON f.ID_Departamento = d.ID_Departamento
GROUP BY
    d.Nome_Departamento
ORDER BY
    Folha_Pagamento_Total DESC;
"""
display(duckdb.query(query).df())

,Nome_Departamento,Numero_de_Funcionarios,Salario_Medio_Anual,Folha_Pagamento_Total
0,Engenharia,4,108750.000000,435000.0
1,Vendas,3,96666.666667,290000.0
2,Marketing,3,78333.333333,235000.0
3,Produto,1,130000.000000,130000.0
4,Recursos Humanos,1,95000.000000,95000.0


### Nível 3: Combinação de Fontes de Dados (JOINS)
**Análise:** Nenhum *dataset* vive isolado. A utilização de `JOIN`s é fundamental para enriquecer os dados dos funcionários com informações contextuais de outras tabelas (departamentos, gerentes e avaliações).

* **Perguntas de Negócio:**
    1.  (Estrutura Organizacional) Qual é o mapa hierárquico da empresa? (Self-Join)
    2.  (Diagnóstico de Processo) Quais funcionários não possuem registro de avaliação em 2023, indicando uma possível lacuna no processo de gestão? (Left Join)

In [ ]:
query = """
SELECT
    f.Nome || ' ' || f.Sobrenome AS Funcionario,
    f.Cargo,
    d.Nome_Departamento AS Departamento,
    g.Nome || ' ' || g.Sobrenome AS Gerente,
    av.Nota_Desempenho AS Nota_2023
FROM
    'Funcionarios.csv' AS f
LEFT JOIN
    'Departamentos.csv' AS d ON f.ID_Departamento = d.ID_Departamento
LEFT JOIN
    'Funcionarios.csv' AS g ON f.ID_Gerente = g.ID_Funcionario -- Self-join no arquivo
LEFT JOIN
    'Avaliacoes_Desempenho.csv' AS av ON f.ID_Funcionario = av.ID_Funcionario AND av.Ano_Avaliacao = 2023
ORDER BY
    Departamento, Funcionario;
"""
display(duckdb.query(query).df())

,Funcionario,Cargo,Departamento,Gerente,Nota_2023
0,Ana Silva,Gerente de Engenharia,Engenharia,None,5
1,Daniel Ferreira,Engenheiro de Software Pleno,Engenharia,Ana Silva,4
2,Elisa Gomes,Engenheira de Software Júnior,Engenharia,Ana Silva,4
3,Fábio Martins,Engenheiro de Software Sênior,Engenharia,Ana Silva,5
4,Carla Dias,Gerente de Marketing,Marketing,None,5
5,Isabela Pereira,Analista de Marketing Digital,Marketing,Carla Dias,4
6,Joana Rodrigues,Estagiária de Marketing,Marketing,Carla Dias,<NA>
7,Maria Souza,Product Manager,Produto,Ana Silva,5
8,Lucas Santos,Analista de RH Sênior,Recursos Humanos,None,4
9,Bruno Costa,Gerente de Vendas,Vendas,None,4


### Nível 4: Análise Temporal (Funções de Data)
**Análise:** A dimensão temporal é crucial em People Analytics. Calcular o tempo de casa (Tenure) dos funcionários nos permite analisar métricas de retenção e senioridade.

* **Perguntas de Negócio:**
    1.  (Análise de Retenção) Quem são os funcionários com maior tempo de casa? Eles ocupam posições estratégicas?
    2.  (Análise de Risco de Turnover) Qual a proporção de funcionários recém-contratados (menos de 2 anos), que podem estar em maior risco de saída?

In [ ]:
# DuckDB usa funções de data padrão do SQL
query = """
SELECT
    Nome,
    Sobrenome,
    Data_Contratacao,
    (CURRENT_DATE - CAST(Data_Contratacao AS DATE)) / 365.25 AS Tempo_de_Casa_Anos
FROM
    'Funcionarios.csv' -- Consultando o arquivo diretamente
ORDER BY
    Tempo_de_Casa_Anos DESC;
"""
display(duckdb.query(query).df())

,Nome,Sobrenome,Data_Contratacao,Tempo_de_Casa_Anos
0,Bruno,Costa,2018-05-01,7.479808
1,Ana,Silva,2019-03-15,6.609172
2,Fábio,Martins,2019-11-05,5.965777
3,Carla,Dias,2020-01-10,5.785079
4,Gabriela,Nunes,2020-08-14,5.190965
5,Lucas,Santos,2020-09-01,5.141684
6,Daniel,Ferreira,2021-02-20,4.670773
7,Isabela,Pereira,2021-05-12,4.449008
8,Maria,Souza,2021-11-15,3.937029
9,Heitor,Oliveira,2022-01-10,3.783710
